In [1]:
import pandas as pd
import numpy as np
orders = pd.read_csv("orders.csv")
items = pd.read_csv("Items_orders.csv")
itemProps = pd.read_csv("Items_attribute.csv",encoding='gbk')
orders_items = pd.merge(orders,items,on="订单编号")
orders_items_props = pd.merge(orders_items,itemProps,on="标题")

In [2]:
result = orders_items_props.loc[:,["买家会员名","宝贝ID"]]
result["购买次数"] = 0
freq = result.groupby(["买家会员名","宝贝ID"]).count().reset_index()
#freq["购买次数"]=1
freq= freq.pivot(index="买家会员名",columns="宝贝ID",values="购买次数")
freqMatrix = freq.fillna(0).values

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
item_similar=cosine_similarity(freqMatrix.T)

In [4]:
#构建预测函数
def predict(similar):
    user_cnt = freqMatrix.shape[0]#用户数3318
    item_cnt = freqMatrix.shape[1]#商品数269
    pred = np.zeros((user_cnt,item_cnt))
    for uid in range(user_cnt):
        for iid in range(item_cnt):
            if freqMatrix[uid,iid] == 0:
                #print (uid,iid)
                pred[uid,iid] = Recommendation_s(uid,iid,similar)
    return pred

#构建一个基于用户和物品的推荐
def Recommendation_s(uid,iid,similar,k=10):
    score = 0
    weight = 0
    uid_action = freqMatrix[uid,:]      #用户uid 对所有商品的行为评分  
    iid_action = freqMatrix[:,iid]      #物品iid 得到的所有用户评分  
    

    iid_sim = similar[iid,:]      #商品iid 对所有商品的相似度    
    sim_indexs = np.argsort(iid_sim)[-(k+1):-1]  #最相似的k个物品的index（除了自己）
    iid_i_mean = np.sum(iid_action)/iid_action[iid_action!=0].size
    for j in sim_indexs :
        if uid_action[j]!=0:
            iid_j_action = freqMatrix[:,j]
            iid_j_mean = np.sum(iid_j_action)/iid_j_action[iid_j_action!=0].size
            score += iid_sim[j]*(uid_action[j]-iid_j_mean)
            weight += abs(iid_sim[j])

    if weight==0:  
        return 0
    else:
        return iid_i_mean + score/float(weight)
    

In [5]:
user_prediction = predict(item_similar)

In [6]:
def get_recom(prediction,k=5):
    recom_df = pd.DataFrame(prediction,columns=freq.columns,index=freq.index)
    
    recom_df = recom_df.stack().reset_index()
    
    recom_df.rename(columns={0:"推荐指数"},inplace=True)
    
    grouped = recom_df.groupby("买家会员名")
   
    topk = grouped.apply(get_topk,k=k)
#    print(topk)
    topk = topk.drop(["买家会员名"],axis=1)
    
    topk.index = topk.index.droplevel(1)
    
    topk.reset_index(inplace=True)
#     print(topk)
    return topk
def get_topk(group,k):
    return group.sort_values("推荐指数",ascending=False)[:k]

In [7]:
user_recom = get_recom(user_prediction,10)
user_recom

,买家会员名,宝贝ID,推荐指数
0,0,544016559367,2.158102
1,0,537396783238,1.945519
2,0,544115359956,1.830060
3,0,546275765548,0.866250
4,0,550715341924,0.783523
5,0,527419046969,0.643857
6,0,536083448292,0.617174
7,0,521019257989,0.565865
8,0,42646163798,0.469041
9,0,547794157516,0.410544
